In [ ]:
!kaggle datasets download ujjwalchowdhury/yoga-pose-classification
!unzip yoga-pose-classification.zip

In [ ]:
import tensorflow as tf

data_directory = 'YogaPoses'

img_hieght = 224,
img_width = 224,
img_size = (224, 224)

batch_size = 32

training_dataset = tf.keras.utils.image_dataset_from_directory(
    data_directory, 
    validation_split = 0.3,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size = batch_size
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)


val_batches = tf.data.experimental.cardinality(validation_dataset)
validation_dataset = validation_dataset.skip(val_batches//2)
test_dataset = validation_dataset.take(val_batches//2)


In [6]:
import pandas as pd
import mediapipe as mp
import numpy as np

landmark_data = []
mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)


# images, labels = next(iter(training_dataset.take(1)))
# result = mp_pose.process(img)
# print(result.pose_landmarks.landmark)

#Extracting Key Point Data to add to csv file. 
key_points_list = []
labels_list = []
nan_count = 0
for images, labels in training_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append( landmarks)
            labels_list.append(label)

df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("training_yoga2_keypoints.csv", index=False)


key_points_list = []
labels_list = []
for images, labels in validation_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append( landmarks)
            labels_list.append(label)

df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("validation_yoga2_keypoints.csv", index=False)

key_points_list = []
labels_list = []
for images, labels in test_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append(landmarks)
            labels_list.append(label)
print(nan_count)
df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("test_yoga2_keypoints.csv", index=False)


I0000 00:00:1755627465.230241 3728992 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1755627465.297190 3750030 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755627465.305347 3750030 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


113
